# Import libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [2]:
import pandas as pd  
import numpy as np
from statsmodels import robust
from pandas import read_excel
from scipy import spatial
from numpy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import mplcursors
import random
from matplotlib import gridspec
import tensorflow as tf
import os
from concurrent.futures import ProcessPoolExecutor
from numba import jit


2024-11-26 11:36:52.804147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 11:36:53.739185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1. Read the data


In [3]:
file_name = "/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained binary MHB Bbox Size 90/MHB_normalized/alldata.xlsx"
#file_name = "/home/gauss/Desktop/Pretrained_Features_ FilteredBbox/O8W_Features_Normalized/alldata.xlsx"
#file_name = "/home/gauss/Desktop/CellProfiler_Features/O8W_normalized.xlsx"
#file_name = "/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained MHB Bbox Size 90/MHB_normalized/alldata.xlsx"
data = pd.read_excel(file_name)
data['Plate number'] = data['Plate number'].apply(str)
data.head()

,Unnamed: 0.1,Plate number,Combination,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1279,feature1280,Category,Compound,Concentration,Image,Label,MoA,Unnamed: 0,Well_number
0,0,1,MHB,1.357533,2.470934,1.857625,1.908892,2.057443,0.917080,0.352288,...,2.084519,-1.517412,Negative Controls,cells,NaN,r01c01ch1.tiff,cells,cells,0,A1
1,1,1,MHB,-8.289253,-21.886908,-10.983758,-9.782533,-16.590643,4.853266,2.391330,...,-9.090403,16.833577,Miscl.,BzCl,0,r01c02ch1.tiff,BzCl,BzCl,1,A2
2,2,1,MHB,-4.223545,-13.179606,-5.147248,-3.934834,-10.358787,2.489697,-5.154178,...,-7.389260,4.072238,HDAC,Vorinostat,10000,r01c03ch1.tiff,Drug,HDAC inhibitor,2,A3
3,3,1,MHB,1.261878,1.889853,1.288764,1.914664,1.793762,-0.004613,1.081149,...,1.834686,-1.153888,Miscl.,Bicalutamide,10000,r01c04ch1.tiff,Drug,Nonsteriodal antiandrogen,3,A4
4,4,1,MHB,0.950500,1.461038,0.928861,1.411623,1.198404,0.123916,1.347225,...,1.616535,-0.895703,Miscl.,Prednisolone,10000,r01c05ch1.tiff,Drug,"Glucocorticoid, immunomodulatory agent",4,A5


# 2. Correlation matrix 

## To calculate enrichment score there are two ways:
### 1. Calculate enrichment for each combination i.e KB, O3B, MHB etc 

In [4]:
#if correlation matrix across entire 8 plates for each combination is required
cat = ["MHB"]
data = data.loc[data['Combination'].isin(cat)]
data

,Unnamed: 0.1,Plate number,Combination,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1279,feature1280,Category,Compound,Concentration,Image,Label,MoA,Unnamed: 0,Well_number
0,0,1,MHB,1.357533,2.470934,1.857625,1.908892,2.057443,0.917080,0.352288,...,2.084519,-1.517412,Negative Controls,cells,NaN,r01c01ch1.tiff,cells,cells,0,A1
1,1,1,MHB,-8.289253,-21.886908,-10.983758,-9.782533,-16.590643,4.853266,2.391330,...,-9.090403,16.833577,Miscl.,BzCl,0,r01c02ch1.tiff,BzCl,BzCl,1,A2
2,2,1,MHB,-4.223545,-13.179606,-5.147248,-3.934834,-10.358787,2.489697,-5.154178,...,-7.389260,4.072238,HDAC,Vorinostat,10000,r01c03ch1.tiff,Drug,HDAC inhibitor,2,A3
3,3,1,MHB,1.261878,1.889853,1.288764,1.914664,1.793762,-0.004613,1.081149,...,1.834686,-1.153888,Miscl.,Bicalutamide,10000,r01c04ch1.tiff,Drug,Nonsteriodal antiandrogen,3,A4
4,4,1,MHB,0.950500,1.461038,0.928861,1.411623,1.198404,0.123916,1.347225,...,1.616535,-0.895703,Miscl.,Prednisolone,10000,r01c05ch1.tiff,Drug,"Glucocorticoid, immunomodulatory agent",4,A5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,375,8,MHB,-1.702203,-1.758454,-0.224354,-0.944739,-0.301446,-0.975748,-0.505870,...,-2.996478,1.802113,Miscl.,BRD7116,10000,r16c20ch1.tiff,Drug,Leukemic stem cell inhibitor,375,P20
3053,376,8,MHB,0.106351,-0.554856,0.846784,2.243682,2.035096,-1.972034,1.948550,...,1.006738,0.563969,Miscl.,JPH203,5000,r16c21ch1.tiff,Drug,LAT1 inhibitor,376,P21
3054,377,8,MHB,0.174647,-0.413930,1.238799,2.296307,2.134409,-1.636411,0.932358,...,1.265476,0.108777,Negative Controls,DMSO,0,r16c22ch1.tiff,DMSO,DMSO,377,P22
3055,378,8,MHB,0.546968,0.177874,1.682488,1.976505,1.991556,-0.764344,0.411934,...,1.146957,-0.292574,Miscl.,RSL3,10000,r16c23ch1.tiff,Drug,"GPX4 inhibitor, induces ferroptosis",378,P23


### 2. Compute correlation matrix across entire dataset and then compute Enrichment

In [5]:
new_data = data.drop(columns=[ "Unnamed: 0","Unnamed: 0.1","Image", "Label"], axis=1) #For neural network
#new_data = data.drop(columns=[ "Unnamed: 0","Unnamed: 0.1", "Label", "FileName_Cancer", "FileName_Cellpose", "FileName_Dapi", "FileName_Fibroblast", "ImageNumber"], axis=1) #"Unnamed: 0.1.1"

new_data['well_annotation'] =  new_data['Plate number']+ '-' + new_data['Combination']+ '-' + new_data['Well_number'] + '-' + new_data['Compound'] +  '-' + new_data['MoA'] 

new_data['well_num'] =  new_data['Plate number'] + '-' + new_data['Combination']+ '-' + new_data['Well_number']


data_new = new_data.drop(columns=["Plate number", "Combination","Concentration","Compound","Well_number", "MoA", "Category", "well_annotation", "well_num"], axis=1)

#prepare dataframe for correlation matrix
data_corr = new_data.drop(columns=["Plate number", "Combination", "Concentration","Compound", "MoA", "Category", "well_annotation", "Well_number"], axis=1)
#Put well_num col as first and set it as index col
first_column = data_corr.pop('well_num')
data_corr.insert(0, 'well_num', first_column)
data_corr.set_index('well_num', inplace=True)

#transpose the dataframe and compute coorelation coefficient
data_new_T = data_corr.T
correlation_mat = data_new_T.corr()
correlation_mat.head()

well_num,1-MHB-A1,1-MHB-A2,1-MHB-A3,1-MHB-A4,1-MHB-A5,1-MHB-A6,1-MHB-A7,1-MHB-A8,1-MHB-A9,1-MHB-A10,...,8-MHB-P15,8-MHB-P16,8-MHB-P17,8-MHB-P18,8-MHB-P19,8-MHB-P20,8-MHB-P21,8-MHB-P22,8-MHB-P23,8-MHB-P24
well_num,,,,,,,,,,,,,,,,,,,,,
1-MHB-A1,1.000000,-0.642035,-0.439204,0.766052,0.579716,-0.419955,-0.297477,-0.249094,-0.706233,-0.746077,...,-0.535446,-0.031449,-0.095219,-0.568544,-0.432219,-0.142656,0.434978,0.512590,0.674723,-0.524443
1-MHB-A2,-0.642035,1.000000,0.499799,-0.571706,-0.387258,0.462486,0.301739,0.716311,0.548978,0.535753,...,0.434701,0.107667,0.395820,0.461115,0.280486,0.134920,-0.429233,-0.437193,-0.515093,0.298419
1-MHB-A3,-0.439204,0.499799,1.000000,-0.425588,-0.336932,0.292696,0.526072,0.379893,0.141649,0.250828,...,0.282922,0.118495,0.313536,0.254136,0.458566,0.152199,-0.335547,-0.328962,-0.346377,0.279277
1-MHB-A4,0.766052,-0.571706,-0.425588,1.000000,0.912318,-0.455443,-0.351723,-0.126350,-0.714661,-0.715238,...,-0.559195,0.008727,-0.348112,-0.626084,-0.426262,-0.171466,0.633691,0.722931,0.722897,-0.567132
1-MHB-A5,0.579716,-0.387258,-0.336932,0.912318,1.000000,-0.344884,-0.281508,0.084806,-0.666418,-0.667954,...,-0.504232,-0.054320,-0.261864,-0.563752,-0.407607,-0.258520,0.440769,0.552842,0.495874,-0.552963


In [6]:
X = data_new.iloc[:,:].values #features on which distance will be calculated
y = new_data["well_annotation"] #well annotation to map the distance index later
z = new_data["Category"]
conc = data["Concentration"] #change to "data" in case using full dataframe
y = y.values
z = z.values
conc = conc.values
'''
print(X)
print(y)
print(z)
print(conc)
'''

'\nprint(X)\nprint(y)\nprint(z)\nprint(conc)\n'

# 4. Calculating enrichment score and Pvalue

In [7]:
well_annot = []
well_conc = []
well_cat = []
highest_es = []
pval = []

@jit(nopython=True)
def compute_res(correlation_matrix, well_category, cat_moa):
    M = np.abs(correlation_matrix[well_category == cat_moa]).sum()
    T = (well_category != cat_moa).sum()
    res = np.zeros(len(correlation_matrix))
    res[0] = correlation_matrix[0] * (1/M)
    for j in range(1, len(correlation_matrix)):
        if well_category[j] == cat_moa:
            res[j] = res[j-1] + (correlation_matrix[j] * (1 / M))
        else:
            res[j] = res[j-1] - (1 / T)
    return res

def process_category(category, new_data, y, z, conc, correlation_mat, cat_moa):
    data_enrch = pd.DataFrame({
        'Well_annotation': y,
        'Well_category': z,
        'Well_concentration': conc,
        'Correlation_matrix': correlation_mat[new_data['well_num'][category]]
    })
    data_enrch = data_enrch.sort_values(by='Correlation_matrix', ascending=False).reset_index(drop=True)
    
    correlation_matrix = data_enrch['Correlation_matrix'].values
    well_category = (data_enrch['Well_category'] == cat_moa).values
    
    res = compute_res(correlation_matrix, well_category, True)
    res_original = res.max()
    
    res_max = []
    for _ in range(1000):
        np.random.shuffle(well_category[1:])
        res = compute_res(correlation_matrix, well_category, True)
        res_max.append(res.max())
    count = np.sum(np.array(res_max) >= res_original)
    p_val = count/1000
    
    return {
        'Well_annotation': data_enrch['Well_annotation'].iloc[0],
        'Well_concentration': data_enrch['Well_concentration'].iloc[0],
        'Well_Category': cat_moa,
        'Highest_ES': res_original,
        'Pvalue': p_val
    }

def main():
    moa = ['EGFR', 'VEGFR', 'PI3K', 'Topoisomerase', 'Mitotic', 'MEK1/2', 'PARP', 'CDK', 'BET', 'HDAC']
    results = []

    with ProcessPoolExecutor() as executor:
        for m in moa:
            print(m)
            categories = new_data.loc[new_data['Category'] == m].index.tolist()
            futures = [executor.submit(process_category, category, new_data, y, z, conc, correlation_mat, m) for category in categories]
            results.extend([future.result() for future in futures])

    data_Pval = pd.DataFrame(results)
    return data_Pval

if __name__ == '__main__':
    data_Pval = main()


EGFR
VEGFR
PI3K
Topoisomerase
Mitotic
MEK1/2
PARP
CDK
BET
HDAC


In [8]:
data_Pval_enriched = data_Pval[data_Pval['Pvalue'] < 0.05]
data_Pval_enriched = data_Pval_enriched['Well_Category'].value_counts().rename_axis('MoA').reset_index(name='value')


In [9]:
data_Pval_enriched.to_excel("/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained binary MHB Bbox Size 90/MHB_normalized/pval_norm_MHB.xlsx")


data_Pval.to_excel("/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained binary MHB Bbox Size 90/MHB_normalized/All_MHB_ES_Pval.xlsx")


#count compounds with 1st and 2nd lowest concentration which are enriched

# Extract the drug name
data_Pval['Drug'] = data_Pval['Well_annotation'].apply(lambda x: x.split('-')[-1])

# Sort by Drug and Well_concentration, then filter for the lowest two concentrations with Pvalue < 0.05
filtered_data = (
    data_Pval
    .sort_values(['Drug', 'Well_concentration'])  # Sort by drug and concentration
    .groupby('Drug')                              # Group by drug
    .head(2)                                      # Take the lowest 2 concentrations per drug
)

# Count the entries with Pvalue < 0.05 in these filtered rows
count_low_pvalue = (filtered_data['Pvalue'] < 0.05).sum()

print(f"Number of drugs with Pvalue < 0.05 in the lowest 2 concentrations: {count_low_pvalue}")
filtered_data.to_excel("/home/gauss/Desktop/Revised_Data_Features/EfficientNetB0 trained binary MHB Bbox Size 90/MHB_normalized/lowest2conc_comp_es_MHB.xlsx")


Number of drugs with Pvalue < 0.05 in the lowest 2 concentrations: 62
